# habspec

> Load, Calibrate, HABSpec json spectra files captured by the HABSpec.

In [ ]:
#| default_exp habspec_data_class

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev

In [ ]:
#| hide
nbdev.nbdev_export()

In [ ]:
#| export
import datetime   as dt
import numpy      as np
import time

In [ ]:
import RedTideProcessing.testing      as testing
import RedTideProcessing.habspec      as hs

In [ ]:
spectra_file, spectra_note = testing.select_test_file( 0 )
print(f'{spectra_file=} ')

spectra_file='/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json' 


### class HABSPEC_CAL_DATA

In [ ]:
#| export
class HABSPEC_CAL_DATA:
  '''
  HABSpec Calibration data class.  This class contains the data necessary to
  calibrate the HABSpec sensor for wavelength, radiance, and dark current.
  '''
  def __init__(self):
    self.create_date          = dt.datetime.utcnow()  # When this class was create.
    self.Notes                = None                  # User entered notes about this calibration.
    self.dark_cal_path        = None                  # Path to HABSpec spectra collection used to generate dark cal data.
    self.solar_cal_fn         = None                  # Filename of any solar calibration spectra used.
    self.cfl_cal_fn           = None                  # Filename of any CFL bulb calibration spectra used.
    self.radiance_cal_path    = None                  # Path to HABSpec spectra collection used to generate radiance cals.
    self.radiance_cal_src_fn  = None                  # Path to radiance source file ( QTH bulb spectra ).
    self.pixel0               = None                  #
    self.pixel1               = None                  #
    self.wavelength0          = None                  #
    self.wavelength1          = None                  # 
    self.scale = {
          # Exp    Scale
              1 : [None],
              2 : [None],
              3 : [None],
              4 : [None],
              6 : [None],
              7 : [None],
              8 : [None],
              9 : [None],
             10 : [None],
             15 : [None],
             20 : [None],
             25 : [None],
             30 : [None],
             35 : [None],
             50 : [None],
             75 : [None],
            100 : [None],
            150 : [None],
            200 : [None],
            250 : [None],
            300 : [None],
            350 : [None],
            400 : [None],
            500 : [None],
            750 : [None],
           1000 : [None]
    }
    self.dark = {                                      # 
          # Exp    Dark
              1 : [None],
              2 : [None],
              3 : [None],
              4 : [None],
              6 : [None],
              7 : [None],
              8 : [None],
              9 : [None],
             10 : [None],
             15 : [None],
             20 : [None],
             25 : [None],
             30 : [None],
             35 : [None],
             50 : [None],
             75 : [None],
            100 : [None],
            150 : [None],
            200 : [None],
            250 : [None],
            300 : [None],
            350 : [None],
            400 : [None],
            500 : [None],
            750 : [None],
           1000 : [None]
           }

  def __str__(self):
    rv = ''
    for k in self.__dict__.keys():
      rv += f'{k:20s}: {self.__dict__[k]}\n'
    return rv

In [ ]:
cal_data = HABSPEC_CAL_DATA()

In [ ]:
cal_data.create_date

datetime.datetime(2023, 8, 3, 21, 50, 11, 949646)

In [ ]:
print(cal_data.__str__())

create_date         : 2023-08-03 21:50:11.949646
Notes               : None
solar_cal_fn        : None
cfl_cal_fn          : None
radiance_cal_path   : None
radiance_cal_src_fn : None
pixel0              : None
pixel1              : None
wavelength0         : None
wavelength1         : None
scale               : {1: [None], 2: [None], 3: [None], 4: [None], 6: [None], 7: [None], 8: [None], 9: [None], 10: [None], 15: [None], 20: [None], 25: [None], 30: [None], 35: [None], 50: [None], 75: [None], 100: [None], 150: [None], 200: [None], 250: [None], 300: [None], 350: [None], 400: [None], 500: [None], 750: [None], 1000: [None]}
dark                : {1: [None], 2: [None], 3: [None], 4: [None], 6: [None], 7: [None], 8: [None], 9: [None], 10: [None], 15: [None], 20: [None], 25: [None], 30: [None], 35: [None], 50: [None], 75: [None], 100: [None], 150: [None], 200: [None], 250: [None], 300: [None], 350: [None], 400: [None], 500: [None], 750: [None], 1000: [None]}



#| hide
### class HAB_Spectra

In [ ]:
#| export
class HAB_Spectra:
  '''
  '''
  Configured  = False                       # Gets set true when first legit spectra is read.
  Calibrated_w= False                       # Gets set True when calibrated for wavelength
  Xpixels     = np.arange( 10 )             # Pixel number array. Gets loaded when a spectra is configured. Ultimately 1280 points.
  wavelengths = np.arange( 10 )             # Wavelength array. Gets loaded when a spectra is configured. Ultimately 1280 points.
  create_time = dt.datetime.utcnow()        # 
  pixel0      = None                        # Lowest spectral pixel number. Filled in when calibrated for wavelength.
  wavelength0 = None                        # Lowest spectral wavelength. Filled in when calibrated for wavelength.
  pixel1      = None                        # Highest spectral pixel number. Filled in when calibrated for wavelength.
  wavelength1 = None                        # Highest spectral wavelength. Filled in when calibrated for wavelength.
  def __init__(self ) -> object:
    self.json_spectra_file_name    = None   # The filename of JSON spectra file.
    self.Lat          = None                # GPS latitude.  GPS data only present after 2022-12.
    self.Lon          = None                # GPS longitude.
    self.altitde_m    = None                # GPS altitude in meters.
    self.n_saturated  = None                # number of saturated pixels in the orginal image.
    self.raw_y_min    = None                # The minimum value of of the summed up intensity values.
    self.remove_bias  = None                # True if y_raw has bad raw_y_min subtracted.
    self.y_average    = None                # User set, True if raw_y values are average vs summed up.
    
    self.DateTime     = None                # Unix datetime value.  Only present after 2022-12.
    self.Exposure     = None                # Sensor spectra exposure time in miliseconds. Only after 2022-12.
    
    self.raw_y        = np.arange( 10 )     # Summed up Intensity values. 
    self.summed_rows  = 0                   # Number of summed up pixels per spectra pixel.
    
  def __str__(self):
    '''
    '''
    rv = ''
    rv = f'{self.Calibrated_w=}\n'
    rv+= f'{self.Xpixels=}\n'
    rv+= f'{self.wavelengths=}\n'
    rv+= f'{self.create_time=}\n'
    rv+= f'{self.pixel0=}\n'
    rv+= f'{self.wavelength0=}\n'
    rv+= f'{self.pixel1=}\n'
    rv+= f'{self.wavelength1=}\n'
    for k in self.__dict__.keys():
      rv += f'{k:20s}: {self.__dict__[k]}\n'
    return rv
    

In [ ]:
hab_spectra = HAB_Spectra()
spectra_cal_w = hs.calibrate_using_2_wavelengths(   
  hab_spectra,
  spectra_file,
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 
)

In [ ]:
display(len(hab_spectra.Xpixels), hab_spectra.Xpixels[0:5])

1280

array([0, 1, 2, 3, 4])

In [ ]:
display(len(hab_spectra.Xpixels),hab_spectra.wavelengths[0:5])

1280

array([403.13862212, 403.51718894, 403.89575576, 404.27432258,
       404.6528894 ])

In [ ]:
print(hab_spectra.__str__())

self.Calibrated_w=True
self.Xpixels=array([   0,    1,    2, ..., 1277, 1278, 1279])
self.wavelengths=array([403.13862212, 403.51718894, 403.89575576, ..., 886.56845161,
       886.94701843, 887.32558525])
self.create_time=datetime.datetime(2023, 7, 25, 15, 8, 23, 869159)
self.pixel0=73
self.wavelength0=430.774
self.pixel1=941
self.wavelength1=759.37
json_spectra_file_name: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json
Lat                 : None
Lon                 : None
altitde_m           : None
n_saturated         : None
raw_y_min           : None
remove_bias         : None
y_average           : None
DateTime            : None
Exposure            : None
raw_y               : [73929 73682 73935 ... 39494 39451 39468]
summed_rows         : 800
Xpixels             : [   0    1    2 ... 1277 1278 1279]
wavelengths         : [403.13862212 403.51718894 403.89575576 ... 886.56845161 886.94701843
 887.32558525]
error_msgs    

#| hide
#### def w_to_pixel()

In [ ]:
#| export
def w_to_pixel(
  hab_spectra_class,   # The HAB_Spectra class.
  w,                   # The wavelength you want the pixel number of.
  debug=False          # Set to True to print internals.
) -> int:              # The pixel
  '''
  Return the pixel index for a given wavelength `w`.  Out of range
  wavelengths cause -1 to be returned.
  '''
  x = hab_spectra_class.wavelengths
  dw  = (x[-1] - x[0])
  dpx = len(x)
  d  = dw/dpx
  px = int((w-x[0]) / d )
  if px < 0: 
    px = -1
  elif w > x[-1]:
    px = -1
    
  if debug:
    print(f'debug. w_to_pixel(). {x=}, {dw=}, {dpx=}, {d=}, {px=}' )

  return px

#| hide
#### def pixel_to_w()

In [ ]:
#| export
def pixel_to_w(
  hab_spectra_class,  # The HAB_Spectra class.
  #x,                 # Array of wavlengths if the sensor.
  pix,                # Pixel number to convert to wavelength.
) -> float:           # The wavelength of the pixel.
  '''
  Return the wavelength of a given pixel index.
  Out of range wavelengths cause -1 to be returned.
  '''
  x = hab_spectra_class.wavelengths
  dw  = (x[-1] - x[0])
  dpx = len(x)
  d  = dw/dpx
  w = px*d + x[0]
  if w < x[0]:
    w = -1.0
  elif w > x[-1]:
    w = -1.0
  return w

test w_to_pixel.

In [ ]:
print('       w0   px        w1      Diff')
for w0 in ( 
  hab_spectra.wavelengths[0], 
  300, 500, 600, 700, 800, 1000,
  hab_spectra.wavelengths[-1] 
):
  px  = w_to_pixel( hab_spectra, w0, debug=False)
  w1  = pixel_to_w( hab_spectra, px)
  dif = w1 - w0
  print(f'{w0:9.3f} {px:4d} {w1:9.3f} {dif:9.3f}' )

       w0   px        w1      Diff
  403.139    0   403.139     0.000
  300.000   -1    -1.000  -301.000
  500.000  256   499.976    -0.024
  600.000  520   599.840    -0.160
  700.000  784   699.703    -0.297
  800.000 1049   799.945    -0.055
 1000.000   -1    -1.000 -1001.000
  887.326 1280   887.326     0.000


|! hide
#### def w_range_pixels():

In [ ]:
#| export
def w_range_pixels( 
  hab_spectra_class,  # The HAB_Spectra class.
  #x,        # A numpy array of wavlengths for each pixel.
  a,        # Starting wavelength.
  b         # Ending wavelength.
) -> list:  # List of pixels cooresponding to `a`:`b` wavelengths.
    '''
    Returns a pixel index list of all pixels between wavelength 'a' and 'b'. 
    '''
    x = hab_spectra_class.wavelengths
    rv = np.where(np.logical_and( x>=a, x<=b) )
    return rv

Test w_range_pixels() between 500 and 530 nanometers.

In [ ]:
# simulate the wavelengths since there is no calibration yet.
hab_spectra.wavelengths = np.arange( 0, 1280 )

test_blue_band_edge = 500    # Starting wavelength
test_red_band_edge  = 520    # Stopping wavelength
test_band_pixels = w_range_pixels( hab_spectra, test_blue_band_edge, test_red_band_edge)

A numpy array is returned containing the pixel locations for each wavelength
between `test_blue_band_edge` and `test_red_band_edge`

In [ ]:
display(test_band_pixels)

(array([500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512,
        513, 514, 515, 516, 517, 518, 519, 520]),)

We can compute numpy statistic on the spectra values within this band.  We will compute the mean, average, and
min and max on the test_spectra.

In [ ]:
( hab_spectra.raw_y[test_band_pixels].mean(),
  np.average( hab_spectra.raw_y[test_band_pixels] ), 
  hab_spectra.raw_y[test_band_pixels].min(), 
  hab_spectra.raw_y[test_band_pixels].max()
)

(140707.38095238095, 140707.38095238095, 137251, 144065)